<br/><h1 align='center'> <font color='blue'>Scrapping the List of Disney Movies from WIKI </font></h1><br/><br/>

This is an end-to-end data science project where we extract the data from internet, store it into our database, do some manipulations using SQL then get the data in ```csv``` format for our data science project.

#### Requirements:
* ```Python 3```
* ```PostgreSQL```
* ```BeautifulSoup``` library
* ```Psycopg2``` library to interact with database 

#### Import the libraries

In [1]:
# import the necessary libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import psycopg2

#### Enter the ```wiki URL```

In [ ]:
try:
    url = input("Enter the Disney Movies WIKI URL: - ")
except:
    url = 'https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films'

#### Read the html file

For web scrapping, we will use ```BeautifulSoup``` library which is an amazing library to work on ```html and lxml``` file. It is not an in-built library so install it using following command:<br/><br/>
```pip install bs4```. <br/><br/>
For more <a href='#'>visit here</a>.

In [ ]:
# get the html & BeautifulSoup object
html = urlopen(url).read()
soup = BeautifulSoup(html, 'html.parser')

<br/>Go and take a look at ```wikipedia page``` of disney movies (<a href='#'>click here</a>)

Let's see print out the ```soup``` object.

In [ ]:
print(soup.prettify())

<br/>We see all movies are in ```table``` form. Extract it with ```.find_all()``` function.

In [ ]:
# get the movies table from soup object
movies_table = soup.find_all('table')
print(movies_table.prettif())

#### ```movies table``` are from table 1 to 10 only

In [ ]:
# let's get the first table and see it how does it look like
first_table = movies_table[1]
type(first_table)
print(first_table.prettify())

### Create the ```movies``` table in ```disney_movies``` database

To store the data which we will extract from wiki source, connect to our database called ```disney_movies```. 
<br/>
First of all create the database. Run the following command in ```psql```.
<br/><br/>
``` postgres=# CREATE DATABASE disney_movies;```

#### Now connect to the database

In [ ]:
# connect with database
try:
    connection = psycopg2.connect(user = 'postgres', # enter your details here
                                 password = '0000!',
                                 host = '127.0.0.1',
                                 port = '5432',
                                 database = 'disney_movies')
    cur = connection.cursor()
    
    # print PostgreSQL connection properties
    print(connection.get_dsn_parameters(),"\n")
    
except (Exception, psycopg2.Error) as error:
    print("ERROR occured while connecting to PostgreSQL", error)   

### Create the ```movies``` table

<br/>Our connection with database was successful. It time to create the table for storage of movies. 

In [ ]:
create_table = '''CREATE TABLE movies(
                    id BIGSERIAL PRIMARY KEY NOT NULL,
                    Type_of_Film VARCHAR(5),
                    Title VARCHAR(150),
                    US_Release DATE,
                    Link VARCHAR(100));'''
cur.execute(create_table)
connection.commit()
print("Table created successfully in PostgreSQL ")

#### Create ```NoneType``` object for matching the link

In [ ]:
doc = '''
<html>
<td>hey</td>
</html>

'''
soup1=BeautifulSoup(doc)
type(soup1.a)

<br/> <h4> Define a function to process the table and get the data</h4>

In [ ]:
def process_table(movie_table_data):
    for index, row in enumerate(movie_table_data):
        if index%4 == 0:
            Type_of_Film = row.get_text(strip=True)
            if Type_of_Film == 'The Mighty Ducks':
                Type_of_Film = 'L'
                index = index - 1
        elif index%4 == 1:
            Title = row.get_text(strip=True)
            if type(row.a) is type(soup1.a):
                Link = 'NoFound'
            else:
                Link = row.a['href']
        elif index%4 == 2:
            US_Release = row.get_text(strip=True)
        else:
            # send data to DataFrame or DATABASE
            insert_query = '''INSERT INTO movies(Type_of_Film, Title, Link, US_Release) VALUES(%s, %s, %s, %s)'''
            record_to_insert = (Type_of_Film, Title, Link, US_Release)
            cur.execute(insert_query, record_to_insert)
            connection.commit()
            print(index//4," Insertion Successful!")

In [ ]:
'''We can see all data are in <td> tag. Lets process it one by one.'''

# get the data from <td> tag
# there are 10 tables. let's do for all
for i in range(1,11):
    
    # let's for now skip the 1990s movies b/c of rowspan problem
    if i == 7:
        continue
    movie_table_data = movies_table[i].find_all('td')
    
    # get data and insert it into database
    process_table(movie_table_data)

### Drop the rows where ```link``` is ```NoFound```

<br/>Some movies have no information on internet. So we will drop those movies.

In [ ]:
drop_query = '''DELETE FROM movies WHERE link = 'NoFound';'''
cur.execute(drop_query)
connection.commit()
print("Successful!")

### We are done with ```postgreSQL``` database. So, let's close it.

In [ ]:
# close database connection
if connection:
    cur.close()
    connection.close()
    print("Connection Closed!")

<br/>
<h1 align='center'><font color='red'>----------------END---------------</font></h1><br/>